In [1]:

#set working directory
setwd("C:\\Users\\Eva\\Google Drive\\Admissions\\NYU\\Admitted\\Courses\\DSGA 3001_9 Responsible DS\\Project\\nutritional-label-for-loan-recommendation-system\\data\\raw")

data <- read.csv("cs-training.csv", header=T)
colnames (data)


[1] "X"                                   
 [2] "SeriousDlqin2yrs"                    
 [3] "RevolvingUtilizationOfUnsecuredLines"
 [4] "age"                                 
 [5] "NumberOfTime30.59DaysPastDueNotWorse"
 [6] "DebtRatio"                           
 [7] "MonthlyIncome"                       
 [8] "NumberOfOpenCreditLinesAndLoans"     
 [9] "NumberOfTimes90DaysLate"             
[10] "NumberRealEstateLoansOrLines"        
[11] "NumberOfTime60.89DaysPastDueNotWorse"
[12] "NumberOfDependents"

In [2]:
##############
# create validation 
##############
#
train <- sample(nrow(data), floor(nrow(data) * 0.66)) 
training <- data[train, ] 
validation <- data[-train, ] 
remove (train)   # remove data to free up space 

In [3]:
# install.packages("DMwR")

In [4]:
#######################
# SMOTE
#######################
# Literature: http://cran.r-project.org/web/packages/DMwR/DMwR.pdf

require(DMwR)
# must set ydata as factor and has to be placed at the end!!!!!!!!
data$ydata <- as.factor(data$ydata)
data <- SMOTE(ydata ~ ., data
                ,k = 5
                ,perc.over = 700,perc.under=200)
table(data$ydata)

Loading required package: DMwR
Warning message:
"package 'DMwR' was built under R version 3.6.3"Loading required package: lattice
Warning message:
"package 'lattice' was built under R version 3.6.3"Loading required package: grid
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


ERROR: Error in `$<-.data.frame`(`*tmp*`, ydata, value = structure(integer(0), .Label = character(0), class = "factor")): replacement has 0 rows, data has 150000


In [8]:
####################################################
#modelling procedure
####################################################

################
#GBM  
################
require(gbm)

model <- gbm(
  ydata ~ ., 
  distribution = "adaboost", 
  data = data, 
  #var.monotone = NULL,
  n.trees = 5000,
  interaction.depth = 4,
  n.minobsinnode = 30,
  shrinkage = 0.05,
  bag.fraction = 0.2,
  train.fraction = 0.8,
  #cv.folds=5,
  #keep.data = TRUE,
  verbose = TRUE)

predict <- predict.gbm (model, test, n.trees = 5000, type = "response")

################
# Random Forest
################
require(randomForest)
data$ydata <- as.factor(data$ydata)

model <- randomForest (ydata ~ ., data, ntree = 250, nodesize = 100)
predict <- predict (model, test, type = "prob")


################
#caret
################
#http://cran.r-project.org/web/packages/caret/caret.pdf
require(caret)
require(MASS)

data$ydata <- as.factor(data$ydata)
model <- train (ydata ~ ., data, method = "xxx")
# replace xxx with a function found in caret

predict <- predict (model, test)

#################
# write output
#################

write.csv(predict, file = "predict.csv", quote = FALSE, row.names = FALSE)

Loading required package: gbm
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"there is no package called 'gbm'"

ERROR: Error in gbm(ydata ~ ., distribution = "adaboost", data = data, n.trees = 5000, : could not find function "gbm"
